In [ ]:
#noetbook utilizado para a aula sobre regressão univariada

In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [4]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("RegressaoLinear").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [5]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
diretorioRegressao="/content/gdrive/MyDrive/Arquivos-MLib/Arquivos/cap4/regressaoLinear.csv"  #diretório que contém o arquivo a ser utilizado

In [7]:
#definindo o esquema dos dados a serem lidos
from pyspark.sql.types import *  #import dos tipos
schema = StructType().add("X", IntegerType(), True).add("Y", StringType(), True)  #define o esquema a ser utilizado


In [8]:
#lendo arquivos armazenados CSV com o esquema definido
pagamentoSeguro = spark.read.format('csv').schema(schema).options(header = 'true',delimiter=';').load(diretorioRegressao)

In [9]:
pagamentoSeguro.printSchema()

root
 |-- X: integer (nullable = true)
 |-- Y: string (nullable = true)



In [10]:
pagamentoSeguro.show(5)

+---+-----+
|  X|    Y|
+---+-----+
|108|392,5|
| 19| 46,2|
| 13| 15,7|
|124|422,2|
| 40|119,4|
+---+-----+
only showing top 5 rows



In [22]:
import pyspark.sql.functions as F
pagamentoSeguro = pagamentoSeguro.select(F.col('X').alias("Apolices"), F.col('Y').alias("Valor_Pago"))  #adiciona nomes ao cabaçalho

AnalysisException: ignored

In [ ]:
pagamentoSeguro.show(5)

+--------+----------+
|Apolices|Valor_Pago|
+--------+----------+
|     108|     392,5|
|      19|      46,2|
|      13|      15,7|
|     124|     422,2|
|      40|     119,4|
+--------+----------+
only showing top 5 rows



In [12]:
pagamentoSeguroPonto = pagamentoSeguro.withColumn("Valor_Pago_Novo", F.regexp_replace(F.col("Valor_Pago"), "[,]", "."))  #troca o valor de "," para "." 

In [13]:
pagamentoSeguroPonto.show(5)

+--------+----------+---------------+
|Apolices|Valor_Pago|Valor_Pago_Novo|
+--------+----------+---------------+
|     108|     392,5|          392.5|
|      19|      46,2|           46.2|
|      13|      15,7|           15.7|
|     124|     422,2|          422.2|
|      40|     119,4|          119.4|
+--------+----------+---------------+
only showing top 5 rows



In [15]:
#modificando o tipo string para numérico
pagamentoSeguroFinal = pagamentoSeguroPonto.select(F.col('Apolices'),F.col('Valor_Pago_Novo'), pagamentoSeguroPonto.Valor_Pago_Novo.cast('float').alias('Valor_Pago_Float'))

In [16]:
pagamentoSeguroFinal.show(5)

+--------+---------------+----------------+
|Apolices|Valor_Pago_Novo|Valor_Pago_Float|
+--------+---------------+----------------+
|     108|          392.5|           392.5|
|      19|           46.2|            46.2|
|      13|           15.7|            15.7|
|     124|          422.2|           422.2|
|      40|          119.4|           119.4|
+--------+---------------+----------------+
only showing top 5 rows



In [17]:
pagamentoSeguroFinal.printSchema()

root
 |-- Apolices: integer (nullable = true)
 |-- Valor_Pago_Novo: string (nullable = true)
 |-- Valor_Pago_Float: float (nullable = true)



Iniciando o Processo de Regressão

In [18]:
pagamentoSeguroFinal.describe().show()

+-------+------------------+-----------------+-----------------+
|summary|          Apolices|  Valor_Pago_Novo| Valor_Pago_Float|
+-------+------------------+-----------------+-----------------+
|  count|                63|               63|               63|
|   mean|22.904761904761905|98.18730158730159|98.18730196877131|
| stddev| 23.35194561605733|87.32755263404981|87.32755386147541|
|    min|                 0|                0|              0.0|
|    max|               124|             98.1|            422.2|
+-------+------------------+-----------------+-----------------+



In [19]:
#transformando os dados (linhas) em vetores
from pyspark.ml.feature import VectorAssembler  #importando a biblioteca responsável por criar o vetor a partir da coluna

assembler = VectorAssembler(inputCols=['Apolices'], outputCol='features')  #define o objeto para transformação
df_seguro = assembler.transform(pagamentoSeguroFinal) #aplica a transformação
df_seguro.printSchema()

root
 |-- Apolices: integer (nullable = true)
 |-- Valor_Pago_Novo: string (nullable = true)
 |-- Valor_Pago_Float: float (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
df_seguro = df_seguro.select(['features','Valor_Pago_Float'])
df_seguro.show(5)

+--------+----------------+
|features|Valor_Pago_Float|
+--------+----------------+
| [108.0]|           392.5|
|  [19.0]|            46.2|
|  [13.0]|            15.7|
| [124.0]|           422.2|
|  [40.0]|           119.4|
+--------+----------------+
only showing top 5 rows



Criando o Modelo de Regressão

In [24]:
from pyspark.ml.regression import LinearRegression  #biblioteca que contém o modelo de regressão

lr = LinearRegression(maxIter=10, labelCol='Valor_Pago_Float') #define o objeto a ser utilizado para regressão
lrModel = lr.fit(df_seguro)

In [ ]:
#Coeficientes angulares e lineares (a e b) da reta de regressão
print(f'Intercepto: {lrModel.intercept}\nCoeficiente Angular: {lrModel.coefficients.values}')

Intercepto: 19.994485035718697
Coeficiente Angular: [3.41382361]


y = 3.4138X + 19.99

In [25]:
#print das estatísticas do modelo
modelsummary = lrModel.summary

print("Variância Explicada:", modelsummary.explainedVariance)
print('R_2: ', modelsummary.r2)
print('Erro médio quadrático: ',modelsummary.meanSquaredError)

modelsummary.residuals.show(5)


Variância Explicada: 6254.31046706825
R_2:  0.8333466721037774
Erro médio quadrático:  1250.7419636077982
+-------------------+
|          residuals|
+-------------------+
| 3.8125652674761454|
| -38.65713283055052|
| -48.67419213440233|
|-21.108600258352624|
| -37.14742784198987|
+-------------------+
only showing top 5 rows



Realizando a Previsão Através do Modelo

In [26]:
modelsummary.predictions.show(5)

+--------+---------------+------------------+--------+------------------+
|Apolices|Valor_Pago_Novo|  Valor_Pago_Float|features|        prediction|
+--------+---------------+------------------+--------+------------------+
|     108|          392.5|             392.5| [108.0]|388.68743473252385|
|      19|           46.2| 46.20000076293945|  [19.0]| 84.85713359348998|
|      13|           15.7|15.699999809265137|  [13.0]| 64.37419194366747|
|     124|          422.2|422.20001220703125| [124.0]| 443.3086124653839|
|      40|          119.4| 119.4000015258789|  [40.0]|156.54742936786877|
+--------+---------------+------------------+--------+------------------+
only showing top 5 rows

